In [175]:
using CSV,  DataFrames
using BenchmarkTools

In [176]:
periodo_anterior(x::Integer) =  x % 100 > 1  ?  x-1  : 12 + (div(x,100) -1) * 100

df = CSV.read("G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/competencia_01_crudo.csv", DataFrame)
size(df)

(981946, 154)

In [178]:
sort!(df, [:numero_de_cliente, :foto_mes])

global periodo_ultimo = maximum( df.foto_mes )
global periodo_anteultimo = periodo_anterior( periodo_ultimo)

# assign most common class values
df.clase_ternaria = @. ifelse( df.foto_mes < periodo_anteultimo, "CONTINUA", missing )

# pre compute sequential time
periodo = @. div(df.foto_mes,100)*12 + df.foto_mes%100

global last = nrow(df)

for i in 1:last
  if df.foto_mes[i] <= periodo_anteultimo &&  i < last &&
        ( df.numero_de_cliente[i] != df.numero_de_cliente[i+1]  ||  df.numero_de_cliente[i] == df.numero_de_cliente[i+1] && periodo[i+1]  > periodo[i] +1)
          df.clase_ternaria[i] = "BAJA+1"
  end

  if df.foto_mes[i] < periodo_anteultimo &&  i+1 < last && df.numero_de_cliente[i] == df.numero_de_cliente[i+1] &&  periodo[i+1] == periodo[i] +1 &&
        ( df.numero_de_cliente[i+1] != df.numero_de_cliente[i+2]  || df.numero_de_cliente[i+1] == df.numero_de_cliente[i+2] && periodo[i+2]  > periodo[i+1] +2)
          df.clase_ternaria[i] = "BAJA+2"
  end
end



BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  750.308 ms … 908.056 ms  ┊ GC (min … max): 1.65% … 17.86%
 Time  (median):     765.542 ms               ┊ GC (median):    1.81%
 Time  (mean ± σ):   801.069 ms ±  73.102 ms  ┊ GC (mean ± σ):  7.01% ±  7.87%

  █                                                           ▃  
  █▁▁▁▁▇▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  750 ms           Histogram: frequency by time          908 ms <

 Memory estimate: 470.33 MiB, allocs estimate: 26650399.

In [174]:
CSV.write( "G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/competencia_01_julia01.csv", df )

"G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/competencia_01_julia01.csv"

# 1. Entrenameinto

In [122]:
using CSV,  DataFrames
#using DecisionTree
using MLJ
Tree = @load DecisionTreeClassifier pkg=DecisionTree
using CategoricalArrays

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\tschoppj\.julia\packages\MLJModels\ziReN\src\loading.jl:159


In [187]:
df = CSV.read("G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/competencia_01_julia.csv", DataFrame)

names(df)

155-element Vector{String}:
 "numero_de_cliente"
 "foto_mes"
 "active_quarter"
 "cliente_vip"
 "internet"
 "cliente_edad"
 "cliente_antiguedad"
 "mrentabilidad"
 "mrentabilidad_annual"
 "mcomisiones"
 ⋮
 "Visa_mpagado"
 "Visa_mpagospesos"
 "Visa_mpagosdolares"
 "Visa_fechaalta"
 "Visa_mconsumototal"
 "Visa_cconsumos"
 "Visa_cadelantosefectivo"
 "Visa_mpagominimo"
 "clase_ternaria"

In [188]:
count_cliente = length(unique(df.numero_de_cliente))

169481

In [189]:
function encontrar_registros_faltantes(df)
    registros_faltantes = []
    grupos = groupby(df, :numero_de_cliente)

    for grupo in grupos
        registros_cliente = grupo.foto_mes
        registros_cliente = sort(registros_cliente)

        if length(registros_cliente) == 6 && all(diff(registros_cliente) .== 1)
            continue
        end
        if length(registros_cliente) == 5 && (registros_cliente[1] == registros_cliente[end] - 4 || registros_cliente[end] == registros_cliente[1] + 4)
            continue
        end
        if any(diff(registros_cliente) .> 1) 
            push!(registros_faltantes, (grupo.numero_de_cliente[1]))
        end
    end

    return registros_faltantes
end

registros_faltantes = encontrar_registros_faltantes(df)
println(registros_faltantes)

@benchmark encontrar_registros_faltantes(df)

Any[263751894, 304051318, 351895410, 445822700, 493399293, 555298549, 559063769, 583196648, 644687585, 675442310, 832767503, 855499233, 885810926, 893574159, 933210746, 1179226843, 1302790375, 1389447490, 1435040696, 1460648156, 1471746344, 1524390423, 1549075943]


BenchmarkTools.Trial: 36 samples with 1 evaluation.
 Range (min … max):   93.619 ms … 231.728 ms  ┊ GC (min … max):  7.37% … 59.00%
 Time  (median):     120.028 ms               ┊ GC (median):    26.29%
 Time  (mean ± σ):   141.345 ms ±  53.870 ms  ┊ GC (mean ± σ):  38.17% ± 21.22%

  █          ▁                                                   
  █▃▁▁▁▁▁▁▁▁▄█▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▄▆▃▁▁▃▃ ▁
  93.6 ms          Histogram: frequency by time          232 ms <

 Memory estimate: 145.84 MiB, allocs estimate: 2540128.

In [190]:
println(length(registros_faltantes))
registros_faltantes = [Int64(x) for x in registros_faltantes]

23


23-element Vector{Int64}:
  263751894
  304051318
  351895410
  445822700
  493399293
  555298549
  559063769
  583196648
  644687585
  675442310
          ⋮
  933210746
 1179226843
 1302790375
 1389447490
 1435040696
 1460648156
 1471746344
 1524390423
 1549075943

In [192]:
# Dividir el DataFrame en entrenamiento y prueba
df_train = filter(row -> row.foto_mes <= 202104, df)
df_test = filter(row -> row.foto_mes == 202106, df)

print("cantidad de registros de Train: ")
println(size(df_train))
# Eliminar los registros de estos clientes del df_train
df_train = df_train[.!in.(df_train.numero_de_cliente, Ref(registros_faltantes)), :]
print("cantidad de registros de Train: ")
println(size(df_train))

#entrenameitno y target
X_train = select(df_train, Not(:clase_ternaria))
y_train = df_train.clase_ternaria


####----------------Control Missing
# Crear un DataFrame para mostrar los resultados
resultadoX_train = DataFrame(columna = String[], valores_faltantes = Int[])


# Iterar sobre cada columna del DataFrame original
for col in names(X_train)
    # Contar los valores faltantes en la columna actual
    missing_count = sum(ismissing.(X_train[!, col]))
    
    # Añadir el resultado al DataFrame de resultados
    push!(resultadoX_train, (col, missing_count))
end

# Contar los valores faltantes en la columna actual
resultadoy_train = sum(ismissing.(y_train))

# Mostrar el resultado
println(resultadoX_train)
print("Valores faltantes en y_train: ")
println(resultadoy_train)

cantidad de registros de Train: (652447, 155)
cantidad de registros de Train: (652383, 155)
154×2 DataFrame
 Row │ columna                            valores_faltantes 
     │ String                             Int64             
─────┼──────────────────────────────────────────────────────
   1 │ numero_de_cliente                                  0
   2 │ foto_mes                                           0
   3 │ active_quarter                                     0
   4 │ cliente_vip                                        0
   5 │ internet                                           0
   6 │ cliente_edad                                       0
   7 │ cliente_antiguedad                                 0
   8 │ mrentabilidad                                      0
   9 │ mrentabilidad_annual                               0
  10 │ mcomisiones                                        0
  11 │ mactivos_margen                                    0
  12 │ mpasivos_margen                           

## Tratameitno de Misiing en X_train

Muy burdo todo por 0 habra que evaluar mucho de esto como DM.

In [193]:
#Remplazo los valores faltantes por 0, burdo pero me deja entrenar el modelo.
X_train = coalesce.(X_train, 0)
y_train = coalesce.(y_train, "Missing")
#se quita la columna numero_de_cliente
X_train = select(X_train, Not(:numero_de_cliente))
#y_train = select(y_train, Not(:numero_de_cliente))

#Pasamos a Matriz para entrenamiento
#X_train = Matrix(X_train)
#y_train = coalesce.(y_train, "CONTINUA")
#y_train = categorical(y_train)

#Preparamos los datos para entrenar como solicita MJL
#X_train_tab = Table(X_train)
X_train_tab = DataFrame(X_train)


# Obtener los niveles de la variable objetivo
levels_y = levels(y_train)
println(levels_y)
# Convertir la variable objetivo en un vector de categorías finitas
y_train_cat_finite = CategoricalArray(y_train, levels=levels_y)


String15["BAJA+1", "BAJA+2", "CONTINUA"]


652383-element CategoricalArray{String15,1,UInt32}:
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 ⋮
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")

In [194]:
# Ajustar parámetros
max_depth = 7
min_samples_split = 250
min_samples_leaf = 100

tree = Tree(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

# Crear una máquina con el modelo y los datos de entrenamiento
mach = machine(tree, X_train, y_train_cat_finite)

# Entrenar el modelo
MLJ.fit!(mach)

┌ Info: Training machine(DecisionTreeClassifier(max_depth = 7, …), …).
└ @ MLJBase C:\Users\tschoppj\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = 7, …)
  args: 
    1:	Source @053 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}
    2:	Source @951 ⏎ AbstractVector{Multiclass{3}}


## Preparar df_test para aplicar el modelo

In [195]:
# Suponiendo que las columnas de características son las mismas que X_train
# y que ya has tratado los valores faltantes en df_test
X_test = select(df_test, Not(:numero_de_cliente, :clase_ternaria))

#Se eliminan missing y se remplazan por 0
X_test = coalesce.(X_test, 0)

# Realizar predicciones en el conjunto de datos de prueba
probabilidades = MLJ.predict(mach, X_test)



164876-element CategoricalDistributions.UnivariateFiniteVector{Multiclass{3}, String15, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000526, BAJA+2=>0.00123, CONTINUA=>0.998)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000534, BAJA+2=>0.000672, CONTINUA=>0.999)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.0, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.00969, BAJA+2=>0.0196, CONTINUA=>0.971)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.014, CONTINUA=>0.986)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000275, BAJA+2=>0.000153, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.00119, CONTINUA=>0.999)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.000312, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000171, BAJA+2=>9.53e-5, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.000119, CONTINUA=>1.0)
 ⋮
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.00153, BAJA+2=>0.0046, CONTINUA=>0.99

In [196]:
# Extraer las probabilidades de la clase positiva (BAJA+2)
probabilidades_positivas = [pdf(prob, "BAJA+2") for prob in probabilidades]

# Establecer el umbral
umbral = 1/40

# Realizar predicciones con el umbral
predicciones_umbral = [prob > umbral ? 1 : 0 for prob in probabilidades_positivas]

print("Predicciones de personas que dejan el Banco: ")
n_unos = sum(predicciones_umbral)
println(n_unos)

Predicciones de personas que dejan el Banco: 8514


In [197]:
# Agregar las predicciones como una nueva columna en df_test
df_test[!, :Predicted] = predicciones_umbral

# Seleccionar las columnas 'numero_de_cliente' y 'Predicted' para la exportación
resultado_exportar = select(df_test, :numero_de_cliente, :Predicted)

# Exportar a CSV
archivo_numero = "002"
CSV.write("G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/exp/KA2001/KJulia_" * archivo_numero * ".csv", resultado_exportar)

"G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/exp/KA2001/KJulia_002.csv"